Introspection is a subjective process for humans, and it is only capable of informing so much. In Cognitive Psychology, we are learning that cognition is broadly created from bottom-up and top-down processes, firstly perceiving the environment, and secondly and often simultaneously imposing our knowledge and experiences on our interpretations of the external and running internal thoughts. A lot of intuition and subconcious components in cognition would be undetectable to introspection.
I would think that it can be useful in informing us about some overarching themese in cognitive processes. 

In [ ]:
## TSP problem formulation
from tools.aima.search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule, genetic_search
from random import randrange
from threading import Timer
import math
import time

debug = 0
class TravelingSalesman(Problem):
    """
    initial: a random complete city circuit.
    map: a dictionary containing the distances between any two cities in the circuit.
    """
    def __init__(self, numCities, initial, map):
        self.num = numCities
        self.initial = initial
        self.map = map

    # actions returns a list of pairs of cities to swap in the current state
    def actions(self, state):
        """
        :param state: the current state containing a circuit
        :return: actions, a list containing tuples of pairs of cities to swap order in the circuit. The list contains all possible swaps
        """
        actions = []
        lastCity = max(state)
        i = 1
        while i < (len(state)-2) :
            currentCity = state[i]
            for selectSwap in range(1, lastCity+1):
                if currentCity != selectSwap:
                    actions.append([currentCity, selectSwap])
            i += 1
        return actions

    def result(self, state, action):
        """
        :param state: current state in the tsp
        :param action: a selected action (pair of cities to swap)
        :return: the new state resulting from swapping the selected pair of cities to the given state
        """
        new_state = state[:]
        swap_1 = state.index(action[0])
        swap_2 = state.index(action[1])
        new_state[swap_1] = action[1]
        new_state[swap_2] = action[0]
        return new_state

    def value(self, state):
        """
        :param state: the current city circuit
        :return: the sum of the travelling distances in the given state, negated in order to reflect value
        distances should be symmetric
        """
        total = 0
        for i in range(1, numCities+1):
            first = state[i-1]
            second = state[i]
            try:
                dist = self.map[(first, second)]
            except:
                dist = self.map[(second, first)]
        # the shortest past is the best, so distances are negated to reflect this.
            total -= dist
            if debug:
                print("current cities: ", first, second, "distance between cities: ", dist)
        return total

## 1.2

Describe your TSP problem formulation.

In [ ]:
print('Here\'s the problem formulation')

Here's the problem formulation


Describe your sample city-domain and how your code does on it.

In [ ]:
print('Here\'s your code to run the TSP problem')

Make some parting comments on this problem.

## 1.3

Follow the same pattern for this problem.

In [1]:
print('Here\'s yet more code')

Here's yet more code
